# SVM IY010_simulation_7 analysis: Varying 2 stats, fixing 1 stat 

In [1]:
from __future__ import annotations
import numpy as np
import pandas as pd
import glob
from pathlib import Path
from typing import Dict, Optional
import sys
import os
import re
from collections import defaultdict
import torch
import time

# Add src directory to Python path
sys.path.append(str(Path.cwd().parent.parent.parent / "src"))

# Import custom modules
from classifiers.svm_classifier import svm_classifier, grid_search_svm
from models.TF_transformer import TFTransformer, ModelCfg
from utils.data_processing import add_binary_labels, add_nearest_neighbour_labels
from utils.standardise_time_series import standardise_time_series
from utils.shuffle_time_series import shuffle_time_series

# Import sklearn modules for SVM
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler

%load_ext autoreload
%autoreload 2

data_7_mu: mu is fixed, varying CV and t_ac

In [9]:
##### Set up directory paths for data loading ######
BASE_DIR = Path.cwd().parent
OUT_DIR = BASE_DIR
SYNTHETIC_DIR = BASE_DIR / "data_7_mu" 
RESULTS_CSV = "IY010_simulation_parameters_7_mu.csv"
results_csv_path = BASE_DIR / RESULTS_CSV
results = pd.read_csv(results_csv_path)
results = results[results["success"]].dropna(
    subset=["mu_observed", "cv_observed", "t_ac_observed"]
)

label_column = "cv_target" 
# label_column = 't_ac_target' # we could equally choose t_ac_target here, as it's variable, but it will give different results 
labelled_results = add_binary_labels(results, label_column)
# labelled_results = add_nearest_neighbour_labels(results, positive_on=label_column)
##### Set up directory paths for data loading ######

##### Use standardise_time_series utility function ######
# Collect all DataFrames and their labels
data_frames = []
labels = []

for i in range(len(results)):
    trajectory_filename = results["trajectory_filename"].values[i]
    DATA_CSV = SYNTHETIC_DIR / trajectory_filename
    data = pd.read_csv(DATA_CSV)
    data_frames.append(data)
    
    # Get the corresponding label from the results csv
    label_value = labelled_results[labelled_results['trajectory_filename'] == trajectory_filename]['label'].iloc[0]
    labels.append(label_value)

# Use the utility function to standardize
labelled_data = standardise_time_series(data_frames, labels=labels, prefix="t_")

print(f"📏 Standardized dataset:")
print(f"   NaN values: {labelled_data.isnull().sum().sum()}")
##### Use standardise_time_series utility function ######

# =========================================================
# Prepare Features and Labels for SVM
# =========================================================
df = labelled_data.copy()
# Extract labels
y = df["label"].values

# Extract features (all columns except 'label')
X = df.drop(columns=["label"]).values

print(f"Data preparation for SVM:")
print(f"  Feature matrix shape: {X.shape}")
print(f"  Labels shape: {y.shape}")
print(f"  Number of classes: {len(np.unique(y))}")
print(f"  Class distribution: {np.bincount(y)}")
print(f"  Memory usage: {X.nbytes / 1024**2:.2f} MB")

# Check for any NaN or infinite values
if np.any(np.isnan(X)):
    print("⚠️  Warning: NaN values detected in features")
if np.any(np.isinf(X)):
    print("⚠️  Warning: Infinite values detected in features")
    
print("✅ Data ready for SVM classification!")

# SVM Parameters (using defaults from svm_classifier function)
SVM_C = 1.0           # Regularization parameter
SVM_GAMMA = 'scale'   # Kernel coefficient 
SVM_KERNEL = 'rbf'    # Kernel type

# Train/test split ratio
TEST_SPLIT = 0.2
RANDOM_STATE = 42

# Split the data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=TEST_SPLIT, 
    random_state=RANDOM_STATE,
    stratify=y  # Ensure balanced split across classes
)

# Record training time
start_time = time.time()

# Train SVM using the imported svm_classifier function
svm_accuracy = svm_classifier(
    X_train, X_test, y_train, y_test,
    svm_C=SVM_C,
    svm_gamma=SVM_GAMMA, 
    svm_kernel=SVM_KERNEL,
    print_classification_report=True,
    print_confusion_matrix=True,
)

training_time = time.time() - start_time
print(f"⏱️  SVM ({SVM_KERNEL}) training and evaluation time: {training_time:.2f} seconds")

# =========================================================
# Experiment: Does Temporal Order Matter for Classification?
# =========================================================
print("\n🔀 Experiment: Shuffling Time Series Data to Test Temporal Order Impact")
# Use the utility function to create shuffled data
df_shuffled = shuffle_time_series(
    df, 
    preserve_columns=['label'], 
    random_state=RANDOM_STATE
)

# Extract features and labels from shuffled data
y_shuffled = df_shuffled["label"].values
X_shuffled = df_shuffled.drop(columns=["label"]).values

# Split the shuffled data
X_train_shuffled, X_test_shuffled, y_train_shuffled, y_test_shuffled = train_test_split(
    X_shuffled, 
    y_shuffled, 
    test_size=TEST_SPLIT,
    random_state=RANDOM_STATE,
    stratify=y_shuffled
)

# Record training time
start_time = time.time()

# Train SVM on shuffled data
svm_accuracy_shuffled = svm_classifier(
    X_train_shuffled, X_test_shuffled, y_train_shuffled, y_test_shuffled,
    svm_C=SVM_C,
    svm_gamma=SVM_GAMMA, 
    svm_kernel=SVM_KERNEL,
    print_classification_report=True,
    print_confusion_matrix=True,
)
training_time = time.time() - start_time
print(f"⏱️  SVM ({SVM_KERNEL}) training and evaluation time: {training_time:.2f} seconds")

📏 Standardized dataset:
   NaN values: 0
Data preparation for SVM:
  Feature matrix shape: (2350, 144)
  Labels shape: (2350,)
  Number of classes: 2
  Class distribution: [1150 1200]
  Memory usage: 2.58 MB
✅ Data ready for SVM classification!
=== SVM (RBF Kernel) Classification Accuracy: 0.79 ===

Classification Report:
              precision    recall  f1-score   support

           0       0.79      0.78      0.78       230
           1       0.79      0.80      0.80       240

    accuracy                           0.79       470
   macro avg       0.79      0.79      0.79       470
weighted avg       0.79      0.79      0.79       470


Confusion Matrix:
[[179  51]
 [ 48 192]]
⏱️  SVM (rbf) training and evaluation time: 0.18 seconds

🔀 Experiment: Shuffling Time Series Data to Test Temporal Order Impact
=== SVM (RBF Kernel) Classification Accuracy: 0.77 ===

Classification Report:
              precision    recall  f1-score   support

           0       0.75      0.77      0.76 

data_7_cv: cv is fixed, varying mu and t_ac

In [14]:
##### Set up directory paths for data loading ######
BASE_DIR = Path.cwd().parent
OUT_DIR = BASE_DIR
SYNTHETIC_DIR = BASE_DIR / "data_7_cv" 
RESULTS_CSV = "IY010_simulation_parameters_7_cv.csv"
results_csv_path = BASE_DIR / RESULTS_CSV
results = pd.read_csv(results_csv_path)
results = results[results["success"]].dropna(
    subset=["mu_observed", "cv_observed", "t_ac_observed"]
)

label_column = "mu_target" 
# label_column = 't_ac_target' # we could equally choose t_ac_target here, as it's variable, but it will give different results 
labelled_results = add_binary_labels(results, label_column)
# labelled_results = add_nearest_neighbour_labels(results, positive_on=label_column)
##### Set up directory paths for data loading ######

##### Use standardise_time_series utility function ######
# Collect all DataFrames and their labels
data_frames = []
labels = []

for i in range(len(results)):
    trajectory_filename = results["trajectory_filename"].values[i]
    DATA_CSV = SYNTHETIC_DIR / trajectory_filename
    data = pd.read_csv(DATA_CSV)
    data_frames.append(data)
    
    # Get the corresponding label from the results csv
    label_value = labelled_results[labelled_results['trajectory_filename'] == trajectory_filename]['label'].iloc[0]
    labels.append(label_value)

# Use the utility function to standardize
labelled_data = standardise_time_series(data_frames, labels=labels, prefix="t_")

print(f"📏 Standardized dataset:")
print(f"   NaN values: {labelled_data.isnull().sum().sum()}")
##### Use standardise_time_series utility function ######

# =========================================================
# Prepare Features and Labels for SVM
# =========================================================
df = labelled_data.copy()
# Extract labels
y = df["label"].values

# Extract features (all columns except 'label')
X = df.drop(columns=["label"]).values

print(f"Data preparation for SVM:")
print(f"  Feature matrix shape: {X.shape}")
print(f"  Labels shape: {y.shape}")
print(f"  Number of classes: {len(np.unique(y))}")
print(f"  Class distribution: {np.bincount(y)}")
print(f"  Memory usage: {X.nbytes / 1024**2:.2f} MB")

# Check for any NaN or infinite values
if np.any(np.isnan(X)):
    print("⚠️  Warning: NaN values detected in features")
if np.any(np.isinf(X)):
    print("⚠️  Warning: Infinite values detected in features")
    
print("✅ Data ready for SVM classification!")

# SVM Parameters (using defaults from svm_classifier function)
SVM_C = 1.0           # Regularization parameter
SVM_GAMMA = 'scale'   # Kernel coefficient 
SVM_KERNEL = 'rbf'    # Kernel type

# Train/test split ratio
TEST_SPLIT = 0.2
RANDOM_STATE = 42

# Split the data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=TEST_SPLIT, 
    random_state=RANDOM_STATE,
    stratify=y  # Ensure balanced split across classes
)

# Record training time
start_time = time.time()

# Train SVM using the imported svm_classifier function
svm_accuracy = svm_classifier(
    X_train, X_test, y_train, y_test,
    svm_C=SVM_C,
    svm_gamma=SVM_GAMMA, 
    svm_kernel=SVM_KERNEL,
    print_classification_report=True,
    print_confusion_matrix=True,
)

training_time = time.time() - start_time
print(f"⏱️  SVM ({SVM_KERNEL}) training and evaluation time: {training_time:.2f} seconds")

# =========================================================
# Experiment: Does Temporal Order Matter for Classification?
# =========================================================
print("\n🔀 Experiment: Shuffling Time Series Data to Test Temporal Order Impact")
# Use the utility function to create shuffled data
df_shuffled = shuffle_time_series(
    df, 
    preserve_columns=['label'], 
    random_state=RANDOM_STATE
)

# Extract features and labels from shuffled data
y_shuffled = df_shuffled["label"].values
X_shuffled = df_shuffled.drop(columns=["label"]).values

# Split the shuffled data
X_train_shuffled, X_test_shuffled, y_train_shuffled, y_test_shuffled = train_test_split(
    X_shuffled, 
    y_shuffled, 
    test_size=TEST_SPLIT,
    random_state=RANDOM_STATE,
    stratify=y_shuffled
)

# Record training time
start_time = time.time()

# Train SVM on shuffled data
svm_accuracy_shuffled = svm_classifier(
    X_train_shuffled, X_test_shuffled, y_train_shuffled, y_test_shuffled,
    svm_C=SVM_C,
    svm_gamma=SVM_GAMMA, 
    svm_kernel=SVM_KERNEL,
    print_classification_report=True,
    print_confusion_matrix=True,
)
training_time = time.time() - start_time
print(f"⏱️  SVM ({SVM_KERNEL}) training and evaluation time: {training_time:.2f} seconds")

📏 Standardized dataset:
   NaN values: 0
Data preparation for SVM:
  Feature matrix shape: (2350, 144)
  Labels shape: (2350,)
  Number of classes: 2
  Class distribution: [1150 1200]
  Memory usage: 2.58 MB
✅ Data ready for SVM classification!
=== SVM (RBF Kernel) Classification Accuracy: 0.79 ===

Classification Report:
              precision    recall  f1-score   support

           0       0.80      0.77      0.79       230
           1       0.79      0.81      0.80       240

    accuracy                           0.79       470
   macro avg       0.79      0.79      0.79       470
weighted avg       0.79      0.79      0.79       470


Confusion Matrix:
[[178  52]
 [ 45 195]]
⏱️  SVM (rbf) training and evaluation time: 0.14 seconds

🔀 Experiment: Shuffling Time Series Data to Test Temporal Order Impact
=== SVM (RBF Kernel) Classification Accuracy: 0.78 ===

Classification Report:
              precision    recall  f1-score   support

           0       0.76      0.80      0.78 

data_7_t_ac: t_ac is fixed, varying mu and cv

In [17]:
##### Set up directory paths for data loading ######
BASE_DIR = Path.cwd().parent
OUT_DIR = BASE_DIR
SYNTHETIC_DIR = BASE_DIR / "data_7_t_ac" 
RESULTS_CSV = "IY010_simulation_parameters_7_t_ac.csv"
results_csv_path = BASE_DIR / RESULTS_CSV
results = pd.read_csv(results_csv_path)
results = results[results["success"]].dropna(
    subset=["mu_observed", "cv_observed", "t_ac_observed"]
)

# label_column = "cv_target" 
label_column = 'mu_target' # we could equally choose mu_target here, as it's variable, but it will give different results 
labelled_results = add_binary_labels(results, label_column)
# labelled_results = add_nearest_neighbour_labels(results, positive_on=label_column)
##### Set up directory paths for data loading ######

##### Use standardise_time_series utility function ######
# Collect all DataFrames and their labels
data_frames = []
labels = []

for i in range(len(results)):
    trajectory_filename = results["trajectory_filename"].values[i]
    DATA_CSV = SYNTHETIC_DIR / trajectory_filename
    data = pd.read_csv(DATA_CSV)
    data_frames.append(data)
    
    # Get the corresponding label from the results csv
    label_value = labelled_results[labelled_results['trajectory_filename'] == trajectory_filename]['label'].iloc[0]
    labels.append(label_value)

# Use the utility function to standardize
labelled_data = standardise_time_series(data_frames, labels=labels, prefix="t_")

print(f"📏 Standardized dataset:")
print(f"   NaN values: {labelled_data.isnull().sum().sum()}")
##### Use standardise_time_series utility function ######

# =========================================================
# Prepare Features and Labels for SVM
# =========================================================
df = labelled_data.copy()
# Extract labels
y = df["label"].values

# Extract features (all columns except 'label')
X = df.drop(columns=["label"]).values

print(f"Data preparation for SVM:")
print(f"  Feature matrix shape: {X.shape}")
print(f"  Labels shape: {y.shape}")
print(f"  Number of classes: {len(np.unique(y))}")
print(f"  Class distribution: {np.bincount(y)}")
print(f"  Memory usage: {X.nbytes / 1024**2:.2f} MB")

# Check for any NaN or infinite values
if np.any(np.isnan(X)):
    print("⚠️  Warning: NaN values detected in features")
if np.any(np.isinf(X)):
    print("⚠️  Warning: Infinite values detected in features")
    
print("✅ Data ready for SVM classification!")

# SVM Parameters (using defaults from svm_classifier function)
SVM_C = 1.0           # Regularization parameter
SVM_GAMMA = 'scale'   # Kernel coefficient 
SVM_KERNEL = 'rbf'    # Kernel type

# Train/test split ratio
TEST_SPLIT = 0.2
RANDOM_STATE = 42

# Split the data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=TEST_SPLIT, 
    random_state=RANDOM_STATE,
    stratify=y  # Ensure balanced split across classes
)

# Record training time
start_time = time.time()

# Train SVM using the imported svm_classifier function
svm_accuracy = svm_classifier(
    X_train, X_test, y_train, y_test,
    svm_C=SVM_C,
    svm_gamma=SVM_GAMMA, 
    svm_kernel=SVM_KERNEL,
    print_classification_report=True,
    print_confusion_matrix=True,
)

training_time = time.time() - start_time
print(f"⏱️  SVM ({SVM_KERNEL}) training and evaluation time: {training_time:.2f} seconds")

# =========================================================
# Experiment: Does Temporal Order Matter for Classification?
# =========================================================
print("\n🔀 Experiment: Shuffling Time Series Data to Test Temporal Order Impact")
# Use the utility function to create shuffled data
df_shuffled = shuffle_time_series(
    df, 
    preserve_columns=['label'], 
    random_state=RANDOM_STATE
)

# Extract features and labels from shuffled data
y_shuffled = df_shuffled["label"].values
X_shuffled = df_shuffled.drop(columns=["label"]).values

# Split the shuffled data
X_train_shuffled, X_test_shuffled, y_train_shuffled, y_test_shuffled = train_test_split(
    X_shuffled, 
    y_shuffled, 
    test_size=TEST_SPLIT,
    random_state=RANDOM_STATE,
    stratify=y_shuffled
)

# Record training time
start_time = time.time()

# Train SVM on shuffled data
svm_accuracy_shuffled = svm_classifier(
    X_train_shuffled, X_test_shuffled, y_train_shuffled, y_test_shuffled,
    svm_C=SVM_C,
    svm_gamma=SVM_GAMMA, 
    svm_kernel=SVM_KERNEL,
    print_classification_report=True,
    print_confusion_matrix=True,
)
training_time = time.time() - start_time
print(f"⏱️  SVM ({SVM_KERNEL}) training and evaluation time: {training_time:.2f} seconds")

📏 Standardized dataset:
   NaN values: 0
Data preparation for SVM:
  Feature matrix shape: (2400, 200)
  Labels shape: (2400,)
  Number of classes: 2
  Class distribution: [1200 1200]
  Memory usage: 3.66 MB
✅ Data ready for SVM classification!
=== SVM (RBF Kernel) Classification Accuracy: 0.84 ===

Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.83      0.84       240
           1       0.84      0.85      0.84       240

    accuracy                           0.84       480
   macro avg       0.84      0.84      0.84       480
weighted avg       0.84      0.84      0.84       480


Confusion Matrix:
[[200  40]
 [ 36 204]]
⏱️  SVM (rbf) training and evaluation time: 0.15 seconds

🔀 Experiment: Shuffling Time Series Data to Test Temporal Order Impact
=== SVM (RBF Kernel) Classification Accuracy: 0.85 ===

Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.85      0.85 